In [1]:
from Bio.Seq import Seq

def reversecomplement(seq):
    return str(Seq(seq).reverse_complement())

def describe_hgvs_change(ref: str, test: str, gapfill_start: int = 1, return_all: bool = False, revcomp: bool = False) -> list | str:
    if revcomp:
        ref = reversecomplement(ref)
        test = reversecomplement(test)

    if ref == test:
        return ["c.="] if return_all else "c.="

    # SNV detection early
    if len(ref) == len(test):
        diffs = [i for i in range(len(ref)) if ref[i] != test[i]]
        if len(diffs) == 1:
            pos = gapfill_start + diffs[0]
            return [f"c.{pos}{ref[diffs[0]]}>{test[diffs[0]]}"] if return_all else f"c.{pos}{ref[diffs[0]]}>{test[diffs[0]]}"

    # Find leftmost and rightmost differences
    left = 0
    while left < len(ref) and left < len(test) and ref[left] == test[left]:
        left += 1

    right_ref = len(ref)
    right_test = len(test)
    while right_ref > left and right_test > left and ref[right_ref - 1] == test[right_test - 1]:
        right_ref -= 1
        right_test -= 1

    ref_segment = ref[left:right_ref]
    test_segment = test[left:right_test]

    pos_left = gapfill_start + left
    pos_right = gapfill_start + right_ref - 1

    hgvs_list = []

    # Deletion
    if ref_segment and not test_segment:
        if return_all:
            # Always include full range del
            if pos_left == pos_right:
                hgvs_list += [
                    f"c.{pos_left}del{ref_segment}",
                    f"c.{pos_left}del",
                    f"c.{pos_left}_{pos_right}del{ref_segment}",
                    f"c.{pos_left}_{pos_right}del"
                ]
            else:
                hgvs_list += [
                    f"c.{pos_left}_{pos_right}del{ref_segment}",
                    f"c.{pos_left}_{pos_right}del"
                ]
            # Homopolymer ambiguous deletions
            if len(ref_segment) > 1 and all(base == ref_segment[0] for base in ref_segment):
                base = ref_segment[0]
                for i in range(len(ref_segment)):
                    pos = gapfill_start + left + i
                    hgvs_list += [f"c.{pos}del{base}", f"c.{pos}del"]
        else:
            if pos_left == pos_right:
                hgvs_list = [f"c.{pos_left}del"]
            else:
                hgvs_list = [f"c.{pos_left}_{pos_right}del"]

    # Insertion
    elif not ref_segment and test_segment:
        insertion_call = f"c.{pos_left}_{pos_left + 1}ins{test_segment}"
        dup_candidates = []

        # Search backward for matching sequence to test_segment
        for i in range(max(0, left - len(test_segment)), left + 1):
            candidate = ref[i:i + len(test_segment)]
            if candidate == test_segment:
                dup_start = gapfill_start + i
                dup_end = dup_start + len(test_segment) - 1
                if dup_start == dup_end:
                    dup = f"c.{dup_start}dup{test_segment}"
                    ins = f"c.{dup_start - 1}_{dup_start}ins{test_segment}"
                else:
                    dup = f"c.{dup_start}_{dup_end}dup{test_segment}"
                    ins = f"c.{dup_start - 1}_{dup_end}ins{test_segment}"
                dup_candidates.append((dup, ins))

        if return_all:
            hgvs_list.append(insertion_call)
            for dup, ins in dup_candidates:
                hgvs_list.extend([
                    dup,
                    dup.replace(f"dup{test_segment}", "dup"),
                    ins
                ])
        else:
            if dup_candidates:
                hgvs_list = [dup_candidates[-1][0]]  # most 3′ dup with base
            else:
                hgvs_list = [insertion_call]

    # Delins
    elif ref_segment and test_segment:
        if pos_left == pos_right:
            hgvs_list = [f"c.{pos_left}delins{test_segment}"]
        else:
            hgvs_list = [f"c.{pos_left}_{pos_right}delins{test_segment}"]

    # Deduplicate and sort
    hgvs_list = sorted(set(hgvs_list), key=lambda x: (int(''.join(filter(str.isdigit, x))), x))

    return hgvs_list if return_all else hgvs_list[-1]


In [2]:
gapfill_from_transcriptome = 'ATCAGG'
gapfill_start = 1635

gapfill = 'ATCAGGG'
print(describe_hgvs_change(gapfill_from_transcriptome,gapfill, gapfill_start=gapfill_start, return_all=False, revcomp=True))  

gapfill = 'ACAGG'
print(describe_hgvs_change(gapfill_from_transcriptome,gapfill, gapfill_start=gapfill_start, return_all=False, revcomp=True))


c.1636dupC
c.1639del


In [3]:
import pandas as pd
manifest = pd.read_csv('/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_16plex_part1_1/BC1_giftwrap/manifest_detailed.tsv',sep='\t')
manifest = manifest.loc[manifest['gapfill_start'].notna()]
manifest = manifest.loc[~manifest['gapfill_start'].str.contains(',')]
manifest = manifest.loc[manifest['name'].str.contains('c.')]
manifest = manifest.loc[manifest['gapfill_from_transcriptome'].notna()]
manifest = manifest.loc[manifest['gap_probe_sequence'].notna()]
manifest = manifest.loc[manifest['gap_probe_sequence'] != manifest['gapfill_from_transcriptome']]

manifest['hgvs_change'] = manifest.apply(
    lambda row: describe_hgvs_change(
        str(row['gapfill_from_transcriptome']) if pd.notnull(row['gapfill_from_transcriptome']) else '',
        str(row['gap_probe_sequence']) if pd.notnull(row['gap_probe_sequence']) else '',
        int(row['gapfill_start']) if pd.notnull(row['gapfill_start']) else gapfill_start,
        return_all=False, revcomp = True
    ),
    axis=1
)

### assess how well this did
manifest['original_hgvs'] = manifest['name'].str.split(' ').str[1].str.replace('_extra','')
(manifest['original_hgvs'] == manifest['hgvs_change']).value_counts()


True     53
False     5
Name: count, dtype: int64

In [4]:
unmatched = manifest.loc[manifest['original_hgvs'] != manifest['hgvs_change']][['name','hgvs_change','original_hgvs','gapfill_from_transcriptome','gap_probe_sequence','gapfill_start']]

In [5]:
unmatched['hgvs_change'] = unmatched.apply(
    lambda row: describe_hgvs_change(
        str(row['gapfill_from_transcriptome']) if pd.notnull(row['gapfill_from_transcriptome']) else '',
        str(row['gap_probe_sequence']) if pd.notnull(row['gap_probe_sequence']) else '',
        int(row['gapfill_start']) if pd.notnull(row['gapfill_start']) else gapfill_start,
        return_all=True, revcomp = True
    ),
    axis=1
)


In [6]:
def extract_matching_hgvs(row):
    hgvs = row['hgvs_change']
    orig = row['original_hgvs']
    if isinstance(hgvs, list):
        matches = [x for x in hgvs if x == orig]
        if matches:
            return matches[0]
    return hgvs

unmatched['hgvs_change'] = unmatched.apply(extract_matching_hgvs, axis=1)
unmatched.loc[unmatched['hgvs_change'] != unmatched['original_hgvs']]

,name,hgvs_change,original_hgvs,gapfill_from_transcriptome,gap_probe_sequence,gapfill_start
7,ASXL1 c.1934dupG,[c.1934_1935insTGGCCCG],c.1934dupG,CCC,CCGGGCCACC,1932
87,NFE2 c.113_115,[c.114G>A],c.113_115,CCT,CTT,113
91,TP53 c.919_920,[c.919delinsAA],c.919_920,GC,GTT,919
